# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [ ]:
import numpy as np

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
import inauguralproject

# Question 1

**Explain how you solve the model**

In [12]:
# code for solving the model (remember documentation and comments)

model = HouseholdSpecializationModelClass()

#a. Create list with values of alpha and sigma
alpha_list = [0.25, 0.5, 0.75]
sigma_list = [0.5,1.0,1.5]

df = pd.DataFrame(columns = pd.Index(alpha_list, name="sigma/alpha"), index = pd.Index(sigma_list, name=""))


for i in alpha_list:
    for j in sigma_list:
        model.par.alpha = i
        model.par.sigma = j
        results = model.solve_discrete()
        ratio = results.HF / results.HM
        df.loc[j,i] = f"{ratio:.2f}"

print(df)
#print(tabulate(df, headers = alpha_list, tablefmt = "fancy_grid")) 



/var/folders/vv/t_bqz0156yz6j6lf7zwmwj7w0000gn/T/ipykernel_43901/2100280222.py:69: RuntimeWarning: invalid value encountered in reciprocal
  utility = np.fmax(Q,1e-8)**(1-par.rho)/(1-par.rho)


sigma/alpha  0.25  0.50  0.75
                             
0.5          0.55  1.00  1.83
1.0          0.31  1.00  3.25
1.5          0.21  1.00  4.67


# Question 2

Explain your code and procedure

In [15]:
# code
#Question 2
import matplotlib.pyplot as plt

model_2 = HouseholdSpecializationModelClass()

log_H_ratio = []
log_w_ratio = []

for wF in model_2.par.wF.vec:
    wF = model_2.par.wF
    wM = model_2.par.wM
    optimum = model_2.solve.discrete()
    log_HFM = np.log(optimum.HF / optimum.HM)
    log_H_ratio = np.append(log_H_ratio, log_HFM)
    log_wF = np.log(wF / wM)
    log_w_ratio = np.append(log_w_ratio , log_wF)


# Code for creating the figure

figure = plt.figure()

ax = fig.add_subplot(1,1,1)
ax.plot(log_w_ratio, log_H_ratio, color="blue")
ax.set_xlabel("log(wF/wM)")
ax.set_ylabel("log(HF/HM)")
ax.set_title("Figure 2: log (HF/HM) against log(wF/wM) for wF in {}")


AttributeError: 'float' object has no attribute 'vec'

# Question 3

Explain your code and procedure

In [4]:
# code

ADD CONCISE CONLUSION.